# Gallery Example: M/Hyper/1 Linear Network

This example demonstrates a linear network with n queues in series:
- **Arrivals**: Exponential (Poisson process)
- **Service**: Hyperexponential with constant SCV across queues
- **Servers**: 1 server per queue
- **Scheduling**: FCFS

The high variability service (SCV=n) propagates through the network.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_mhyp1_linear(n=2, Umax=0.9): Network {
    """Create M/Hyp/1-Linear queueing network"""
    val model = Network("M/Hyp/1-Linear")
// Block 1: nodes
    val nodes = []
    val source = Source(model, "mySource")
    nodes.append(source)
    
    for i in range(n):
        val queue = Queue(model, f"Queue{i+1}", SchedStrategy.FCFS)
        nodes.append(queue)
    
    val sink = Sink(model, "mySink")
    nodes.append(sink)
// Block 2: classes
    val oclass = OpenClass(model, "myClass")
// Exponential arrivals
    source.setArrival(oclass, Exp(1))
// Create pyramid utilization pattern
    val half = n // 2
    val means = list(np.linspace(0.1, Umax, half))
    if n % 2 == 0:
        val means = means + means[::-1]
    else:
        val means = means + [Umax] + means[::-1]
// Set HyperExp service for each queue (constant SCV=n)
    for i in range(n):
        nodes[i+1].setService(oclass, HyperExp.fit_mean_and_scv(means[i], n))
// Block 3: topology
    model.link(Network.serialRouting(*nodes))
    
    return model
// Create the model with 2 queues
val model = gallery_mhyp1_linear(n=2)

## About M/Hyper/1 Linear Networks

This model features:
- **Poisson arrivals** (SCV = 1): Standard random arrivals
- **Hyperexponential service** (SCV = n): High variability

With hyperexponential service:
- Some jobs get very fast service
- Others require much longer
- This creates bursts of congestion

The high service variability combined with the linear topology can create significant queue buildup, especially at the bottleneck queue.

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)